# Sentiment Analysis terhadap Vaksinisasi COVID-19 di Indonesia: Data Cleaning
<h2>Tim Yaudahlah</h2>


---

Kaenova Mahendra Auditama<sup>1</sup><br>
Fendi Irfan Amorokhman<sup>2</sup><br>
Ananda Affan Fattahila<sup>3</sup><br>
<sup>1</sup><a href="mailto:kaenova@student.telkomuniversity.ac.id">kaenova@student.telkomuniversity.ac.id</a><br>
<sup>2</sup><a href="mailto:fendiirfan@student.telkomuniversity.ac.id">fendiirfan@student.telkomuniversity.ac.id</a><br>
<sup>3</sup><a href="mailto:affanfattahila@student.telkomuniversity.ac.id">affanfattahila@student.telkomuniversity.ac.id</a><br>
Informatics Engineering, Telkom University, Indonesia<br>
2021


# Installing Requirements

In [1]:
!pip install pandas numpy tqdm

# Connect to Google Drive

In [2]:
# Optional
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Initialize DataCleaning Class

In [3]:
# # #
# Referenceok je joke kasar ni
# 1. https://github.com/datascienceid/stopwords-bahasa-indonesia
# 2. https://github.com/nasalsabila/kamus-alay
# 3. https://www.kaggle.com/rtatman/english-word-frequency
# # # 

import pandas as pd
import re
import string
from tqdm import tqdm

class DataCleaning:
    custom_delete_string = ['malaysia', 'je', 'takde', 'kerajaan', 'ni', 'cakap', 'takda', 'ambik', 'kuala', 'xnak', 'nk', 'klia', 'elok', 'pulak', 'tgk', 'tengok',
                            'amek', 'atok', 'amik', 'nak', 'amp', 'tetibe']

    kamus_alay = {'rpp':"" , 'pjj': "pembelajaran jarak jauh" , "jan" : "jangan" , "emg" : "emang" ,
             "y" : "ya", "lu" : "kamu" , "gue" : "saya" , "gua" : "saya" ,"bkn" : "bukan" , "mksudny" : "maksudnya",
             "gk" : "engga" , "gni" : "gini", "klihatan" : "kelihatan" , "trus" : "terus" ,"beraaa" : "berasa" ,
             "bgt" : "banget","tpi":"tapi","onlen" : "online" ,"yoi" : "ya" , "yooii" : "ya" , "betulll" :"betul",
             "yutub":"youtube" ,"twitt" :"twitter", "bisaa" : "bisa" , "dripada" : "dari pada" , "utk" : "untuk",
             "gegara" :"gara-gara", "donlot" : "mengunduh" ,"w" : "aku" , "dah" : "sudah" , "naek" : "naik",
             "abis" : "habis" , "otw" : "menuju" , "tp" : "tapi" , "bsk" : "besok" , "lg" : "lagi" , "rakan2" : "rekan-rekan" ,
             "mengharungi" : "mengarungi" , "fizikal" : 'fisik',"mangstap" : "mantap" , "bat" : "bener" , "bet" : 'bener',
             "mangat" : "semangat","yg":"yang","sadgurl" : 'perempuan' , "smbil" : "sambil","barbar" : "biadab",
             "ajigggg" : "anjing" , "apaan" : "apa" , "mlooo" : "mulu" , "grgr" : "gara-gara","smt":"semesterr","ttp" :"tetap" ,
             "yok" : "yuk", "beritane" : "beritanya","nurut2" : "nurut-nurut" ,"wa" : 'wassap', 
             "mapel" : "mata pelajaran","ni" : "ini" , "kek" : "kayak","bosen" : "bosan" , "capek" : "cape" ,
             "bego" : "bodoh" , "kagak" : "ngga" , "kesel" : "kesal", "bang" : "mas","ak":"aku" ,"sma" : "sekolah menengah atas",
             "tbtb" :"tiba-tiba","bgt" : "banget","wasaf" : "wassap" ,"anjir":"anjing","ngak" :"nggak",
             "jan" : "jangan","pen" : "pengen" , "offline" : "oflen" , "dasar2nya" : "dasar-dasar nya" , "walhasil" :"hasilnya",
             "semuga" : "semoga","tpi" : "tapi" , "terus2an" : "terus-terus an" , "gaes" : "teman-teman",
             "cm" : "cuma","babay":"selamat tinggal","ntah" : "entah","diruma":"dirumah" , "gtau" : "gatau","anjai" : "anjing",
             "g" : "ngga" , "ad" : "ada" , "samsek" : "sama sekali" , "ppt" : "slide presentasi", "gada" : "ngga ada",
             "quiz" : 'kuis' , "syalan" : "sialan", "puyeng" : "pusing" , "receh" :"tidak menarik","krn":"karena",
             "bget" : "banget" , "quota" : "kouta" , "e learning" : "pembelajaran jarak jauh", "mtk" : "matematika",
             "ansos" : "anti sosial","yutup" : "youtube" , "masalh" : "masalah" , "game" : "permainan","sy":"saya",
             "sprt" : "seperti" , "ketrbtsn" : "keterbatasan" , "kyk" : "seperti" ,"kayak":"seperti","tjukup" : "cukup",
             "bs":"bisa" , "stay" : "menetap" , "cri" : "cari" , "wa" : "wassap" , "ttng" : "tentang","ajg" : "anjing",'sekolah2':'sekolah sekolah',"aing" : "aku","sama2" : "sama-sama" , "kea": "seperti","ato" : "atau","demen":"suka",
             "f2f" : "tatap muka","lgsng" : "langsung","ngirim" : "mengirim","smpe" : "sampai" , "typo" : "salah penulisan",
             "krna" :"karena", "cma" : "cuma","tak" : "tidak","met" :"selamat","miskom" :"salah komunikasi","mager" : "malas",
             "aq":"aku","pgn":"pengen","gada" : "engga ada","mumet":'pusing',
             "@": "di", "abis": "habis", "ad": "ada", "adlh": "adalah", "afaik": "as far as i know",
             "ahaha": "haha", "aj": "saja", "ajep-ajep": "dunia gemerlap", "ak": "saya", "akika": "aku",
             "akkoh": "aku", "akuwh": "aku", "alay": "norak", "alow": "halo", "ambilin": "ambilkan",
             "ancur": "hancur", "anjrit": "anjing", "anter": "antar", "ap2": "apa-apa", "apasih": "apa sih",
             "apes": "sial", "aps": "apa", "aq": "saya", "aquwh": "aku", "asbun": "asal bunyi", "aseekk": "asyik",
             "asekk": "asyik", "asem": "asam", "aspal": "asli tetapi palsu", "astul": "asal tulis", "ato": "atau",
             "au ah": "tidak mau tahu", "awak": "saya", "ay": "sayang", "ayank": "sayang", "b4": "sebelum",
             "bakalan": "akan", "bandes": "bantuan desa", "bangedh": "banget", "banpol": "bantuan polisi",
             "banpur": "bantuan tempur", "basbang": "basi", "bcanda": "bercanda", "bdg": "bandung",
             "begajulan": "nakal", "beliin": "belikan", "bencong": "banci", "bentar": "sebentar"
            , "ber3": "bertiga", "beresin": "membereskan", "bete": "bosan", "beud": "banget",
             "bg": "abang", "bgmn": "bagaimana", "bgt": "banget", "bijimane": "bagaimana",
             "bintal": "bimbingan mental", "bkl": "akan", "bknnya": "bukannya", "blegug": "bodoh",
             "blh": "boleh", "bln": "bulan", "blum": "belum", "bnci": "benci", "bnran": "yang benar",
             "bodor": "lucu", "bokap": "ayah", "boker": "buang air besar", "bokis": "bohong",
             "boljug": "boleh juga", "bonek": "bocah nekat", "boyeh": "boleh", "br": "baru",
             "brg": "bareng", "bro": "saudara laki-laki", "bru": "baru", "bs": "bisa",
             "bsen": "bosan", "bt": "buat", "btw": "ngomong-ngomong", "buaya": "tidak setia",
             "bubbu": "tidur", "bubu": "tidur", "bumil": "ibu hamil", "bw": "bawa", "bwt": "buat",
             "byk": "banyak", "byrin": "bayarkan", "cabal": "sabar", "cadas": "keren", "calo": "makelar",
             "can": "belum", "capcus": "pergi", "caper": "cari perhatian", "ce": "cewek", "cekal": "cegah tangkal",
             "cemen": "penakut", "cengengesan": "tertawa", "cepet": "cepat", "cew": "cewek", "chuyunk": "sayang",
             "cimeng": "ganja", "cipika cipiki": "cium pipi kanan cium pipi kiri", "ciyh": "sih", "ckepp": "cakep",
             "ckp": "cakep", "cmiiw": "correct me if i'm wrong", "cmpur": "campur", "cong": "banci", "conlok": "cinta lokasi",
             "cowwyy": "maaf", "cp": "siapa", "cpe": "capek", "cppe": "capek", "cucok": "cocok", "cuex": "cuek", "cumi": "Cuma miscall",
             "cups": "culun", "curanmor": "pencurian kendaraan bermotor", "curcol": "curahan hati colongan", "cwek": "cewek",
             "cyin": "cinta", "d": "di", "dah": "deh", "dapet": "dapat", "de": "adik", "dek": "adik", "demen": "suka",
             "deyh": "deh", "dgn": "dengan", "diancurin": "dihancurkan", "dimaafin": "dimaafkan", "dimintak": "diminta",
             "disono": "di sana", "dket": "dekat", "dkk": "dan kawan-kawan", "dll": "dan lain-lain", "dlu": "dulu",
             "dngn": "dengan", "dodol": "bodoh", "doku": "uang", "dongs": "dong", "dpt": "dapat", "dri": "dari",
             "drmn": "darimana", "drtd": "dari tadi", "dst": "dan seterusnya", "dtg": "datang", "duh": "aduh",
             "duren": "durian", "ed": "edisi", "egp": "emang gue pikirin", "eke": "aku", "elu": "kamu",
             "emangnya": "memangnya", "emng": "memang", "endak": "tidak", "enggak": "tidak", "envy": "iri",
             "ex": "mantan", "fax": "facsimile", "fifo": "first in first out", "folbek": "follow back",
             "fyi": "sebagai informasi", "gaada": "tidak ada uang", "gag": "tidak", "gaje": "tidak jelas",
             "gak papa": "tidak apa-apa", "gan": "juragan", "gaptek": "gagap teknologi", "gatek": "gagap teknologi",
             "gawe": "kerja", "gbs": "tidak bisa", "gebetan": "orang yang disuka", "geje": "tidak jelas",
             "gepeng": "gelandangan dan pengemis", "ghiy": "lagi", "gile": "gila", "gimana": "bagaimana",
             "gino": "gigi nongol", "githu": "gitu", "gj": "tidak jelas", "gmana": "bagaimana", "gn": "begini",
             "goblok": "bodoh", "golput": "golongan putih", "gowes": "mengayuh sepeda", "gpny": "tidak punya",
             "gr": "gede rasa", "gretongan": "gratisan", "gtau": "tidak tahu", "gua": "saya", "guoblok": "goblok", 
             "gw": "saya", "ha": "tertawa", "haha": "tertawa", "hallow": "halo", "hankam": "pertahanan dan keamanan",
             "hehe": "he", "helo": "halo", "hey": "hai", "hlm": "halaman", "hny": "hanya", "hoax": "isu bohong",
             "hr": "hari", "hrus": "harus", "hubdar": "perhubungan darat", "huff": "mengeluh", "hum": "rumah",
             "humz": "rumah", "ilang": "hilang", "ilfil": "tidak suka", "imho": "in my humble opinion",
             "imoetz": "imut", "item": "hitam", "itungan": "hitungan", "iye": "iya", "ja": "saja",
             "jadiin": "jadi", "jaim": "jaga image", "jayus": "tidak lucu", "jdi": "jadi", "jem": "jam",
             "jga": "juga", "jgnkan": "jangankan", "jir": "anjing", "jln": "jalan", "jomblo": "tidak punya pacar",
             "jubir": "juru bicara", "jutek": "galak", "k": "ke", "kab": "kabupaten", "kabor": "kabur", "kacrut": "kacau",
             "kadiv": "kepala divisi", "kagak": "tidak", "kalo": "kalau", "kampret": "sialan",
             "kamtibmas": "keamanan dan ketertiban masyarakat", "kamuwh": "kamu", "kanwil": "kantor wilayah",
             "karna": "karena", "kasubbag": "kepala subbagian", "katrok": "kampungan", "kayanya": "kayaknya",
             "kbr": "kabar", "kdu": "harus", "kec": "kecamatan", "kejurnas": "kejuaraan nasional",
             "kekeuh": "keras kepala", "kel": "kelurahan", "kemaren": "kemarin", "kepengen": "mau",
             "kepingin": "mau", "kepsek": "kepala sekolah", "kesbang": "kesatuan bangsa",
             "kesra": "kesejahteraan rakyat", "ketrima": "diterima", "kgiatan": "kegiatan",
             "kibul": "bohong", "kimpoi": "kawin", "kl": "kalau", "klianz": "kalian",
             "kloter": "kelompok terbang", "klw": "kalau", "km": "kamu", "kmps": "kampus",
             "kmrn": "kemarin", "knal": "kenal", "knp": "kenapa", "kodya": "kota madya",
             "komdis": "komisi disiplin", "komsov": "komunis sovyet", "kongkow": "kumpul bareng teman-teman"
             , "kopdar": "kopi darat", "korup": "korupsi", "kpn": "kapan", "krenz": "keren", "krm": "kirim",
             "kt": "kita", "ktmu": "ketemu", "ktr": "kantor", "kuper": "kurang pergaulan", "kw": "imitasi",
             "kyk": "seperti", "la": "lah", "lam": "salam", "lamp": "lampiran", "lanud": "landasan udara",
             "latgab": "latihan gabungan", "lebay": "berlebihan", "leh": "boleh", "lelet": "lambat",
             "lemot": "lambat", "lgi": "lagi", "lgsg": "langsung", "liat": "lihat",
             "litbang": "penelitian dan pengembangan", "lmyn": "lumayan", "lo": "kamu",
             "loe": "kamu", "lola": "lambat berfikir", "louph": "cinta", "low": "kalau", 
             "lp": "lupa", "luber": "langsung, umum, bebas, dan rahasia", "luchuw": "lucu", 
             "lum": "belum", "luthu": "lucu", "lwn": "lawan", "maacih": "terima kasih", 
             "mabal": "bolos", "macem": "macam", "macih": "masih", "maem": "makan",
             "magabut": "makan gaji buta", "maho": "homo", "mak jang": "kaget", 
             "maksain": "memaksa", "malem": "malam", "mam": "makan", "maneh": "kamu", 
             "maniez": "manis", "mao": "mau", "masukin": "masukkan", "melu": "ikut",
             "mepet": "dekat sekali", "mgu": "minggu", "migas": "minyak dan gas bumi",
             "mikol": "minuman beralkohol", "miras": "minuman keras", "mlah": "malah", 
             "mngkn": "mungkin", "mo": "mau", "mokad": "mati", "moso": "masa", "mpe": "sampai",
             "msk": "masuk", "mslh": "masalah", "mt": "makan teman", "mubes": "musyawarah besar",
             "mulu": "melulu", "mumpung": "selagi", "munas": "musyawarah nasional",
             "muntaber": "muntah dan berak", "musti": "mesti", "muupz": "maaf",
             "mw": "now watching", "n": "dan", "nanam": "menanam", "nanya": "bertanya",
             "napa": "kenapa", "napi": "narapidana", "napza": "narkotika, alkohol,psikotropika, dan zat adiktif ",
             "narkoba": "narkotika, psikotropika, dan obat terlarang",
             "nasgor": "nasi goreng", "nda": "tidak",
             "ndiri": "sendiri", "ne": "ini", "nekolin": "neokolonialisme", 
             "nembak": "menyatakan cinta", "ngabuburit": "menunggu berbuka puasa", 
             "ngaku": "mengaku", "ngambil": "mengambil", "nganggur": "tidak punya pekerjaan",
             "ngapah": "kenapa", "ngaret": "terlambat", "ngasih": "memberikan",
             "ngebandel": "berbuat bandel", "ngegosip": "bergosip", "ngeklaim": "mengklaim", 
             "ngeksis": "menjadi eksis", "ngeles": "berkilah", "ngelidur": "menggigau",
             "ngerampok": "merampok", "ngga": "tidak", "ngibul": "berbohong", "ngiler": "mau",
             "ngiri": "iri", "ngisiin": "mengisikan", "ngmng": "bicara", "ngomong": "bicara",
             "ngubek2": "mencari-cari", "ngurus": "mengurus", "nie": "ini", "nih": "ini", "niyh": "nih",
             "nmr": "nomor", "nntn": "nonton", "nobar": "nonton bareng", "np": "now playing", "ntar": "nanti",
             "ntn": "nonton", "numpuk": "bertumpuk", "nutupin": "menutupi", "nyari": "mencari", "nyekar": "menyekar",
             "nyicil": "mencicil", "nyoblos": "mencoblos", "nyokap": "ibu", "ogah": "tidak mau", "ol": "online",
             "ongkir": "ongkos kirim", "oot": "out of topic", "org2": "orang-orang", "ortu": "orang tua",
             "otda": "otonomi daerah", "otw": "on the way, sedang di jalan", "pacal": "pacar", "pake": "pakai", 
             "pala": "kepala", "pansus": "panitia khusus", "parpol": "partai politik", "pasutri": "pasangan suami istri", 
             "pd": "pada", "pede": "percaya diri", "pelatnas": "pemusatan latihan nasional", "pemda": "pemerintah daerah",
             "pemkot": "pemerintah kota", "pemred": "pemimpin redaksi", "penjas": "pendidikan jasmani", 
             "perda": "peraturan daerah", "perhatiin": "perhatikan", "pesenan": "pesanan", "pgang": "pegang", 
             "pi": "tapi", "pilkada": "pemilihan kepala daerah", "pisan": "sangat", "pk": "penjahat kelamin", 
             "plg": "paling", "pmrnth": "pemerintah", "polantas": "polisi lalu lintas", "ponpes": "pondok pesantren",
             "pp": "pulang pergi", "prg": "pergi", "prnh": "pernah", "psen": "pesan", "pst": "pasti", "pswt": "pesawat",
             "pw": "posisi nyaman", "qmu": "kamu", "rakor": "rapat koordinasi", "ranmor": "kendaraan bermotor", "re": "reply",
             "ref": "referensi", "rehab": "rehabilitasi", "rempong": "sulit", "repp": "balas",
             "restik": "reserse narkotika", "rhs": "rahasia", "rmh": "rumah", "ru": "baru",
             "ruko": "rumah toko", "rusunawa": "rumah susun sewa", "ruz": "terus", "saia": "saya",
             "salting": "salah tingkah", "sampe": "sampai", "samsek": "sama sekali", "sapose": "siapa",
             "satpam": "satuan pengamanan", "sbb": "sebagai berikut", "sbh": "sebuah", "sbnrny": "sebenarnya",
             "scr": "secara", "sdgkn": "sedangkan", "sdkt": "sedikit", "se7": "setuju", "sebelas dua belas": "mirip",
             "sembako": "sembilan bahan pokok", "sempet": "sempat", "sendratari": "seni drama tari", "sgt": "sangat", 
             "shg": "sehingga", "siech": "sih", "sikon": "situasi dan kondisi", "sinetron": "sinema elektronik", 
             "siramin": "siramkan", "sj": "saja", "skalian": "sekalian", "sklh": "sekolah", "skt": "sakit",
             "slesai": "selesai", "sll": "selalu", "slma": "selama", "slsai": "selesai", "smpt": "sempat",
             "smw": "semua", "sndiri": "sendiri", "soljum": "sholat jumat", "songong": "sombong",
             "sory": "maaf", "sosek": "sosial-ekonomi", "sotoy": "sok tahu", "spa": "siapa",
             "sppa": "siapa", "spt": "seperti", "srtfkt": "sertifikat", "stiap": "setiap", 
             "stlh": "setelah", "suk": "masuk", "sumpek": "sempit", "syg": "sayang",
             "t4": "tempat", "tajir": "kaya", "tau": "tahu", "taw": "tahu", "td": "tadi", "tdk": "tidak",
             "teh": "kakak perempuan", "telat": "terlambat", "telmi": "telat berpikir", "temen": "teman",
             "tengil": "menyebalkan", "tepar": "terkapar", "tggu": "tunggu", "tgu": "tunggu", "thankz": "terima kasih",
             "thn": "tahun", "tilang": "bukti pelanggaran", "tipiwan": "TvOne", "tks": "terima kasih", "tlp": "telepon",
             "tls": "tulis", "tmbah": "tambah", "tmen2": "teman-teman", "tmpah": "tumpah", "tmpt": "tempat", "tngu": "tunggu",
             "tnyta": "ternyata", "tokai": "tai", "toserba": "toko serba ada", "tpi": "tapi", "trdhulu": "terdahulu",
             "trima": "terima kasih", "trm": "terima", "trs": "terus", "trutama": "terutama", "ts": "penulis",
             "tst": "tahu sama tahu", "ttg": "tentang", "tuch": "tuh", "tuir": "tua", "tw": "tahu", "u": "kamu",
             "ud": "sudah", "udah": "sudah", "ujg": "ujung", "ul": "ulangan", "unyu": "lucu", "uplot": "unggah", 
             "urang": "saya", "usah": "perlu", "utk": "untuk", "valas": "valuta asing", "w/": "dengan", "wadir": "wakil direktur", 
             "wamil": "wajib militer", "warkop": "warung kopi", "warteg": "warung tegal", "wat": "buat", "wkt": "waktu",
             "wtf": "what the fuck", "xixixi": "tertawa", "ya": "iya", "yap": "iya", "yaudah": "ya sudah", "yawdah": "ya sudah",
             "yg": "yang", "yl": "yang lain", "yo": "iya", "yowes": "ya sudah", "yup": "iya", "7an": "tujuan", 
             "ababil": "abg labil", "acc": "accord", "adlah": "adalah", "adoh": "aduh", "aha": "tertawa", "aing": "saya",
             "aja": "saja", "ajj": "saja", "aka": "dikenal juga sebagai", "akko": "aku", "akku": "aku", "akyu": "aku", 
             "aljasa": "asal jadi saja", "ama": "sama", "ambl": "ambil", "anjir": "anjing", "ank": "anak", "ap": "apa", 
             "apaan": "apa", "ape": "apa", "aplot": "unggah", "apva": "apa", "aqu": "aku", "asap": "sesegera mungkin", 
             "aseek": "asyik", "asek": "asyik", "aseknya": "asyiknya", "asoy": "asyik", "astrojim": "astagfirullahaladzim", 
             "ath": "kalau begitu", "atuh": "kalau begitu", "ava": "avatar", "aws": "awas", "ayang": "sayang", "ayok": "ayo", 
             "bacot": "banyak bicara", "bales": "balas", "bangdes": "pembangunan desa", "bangkotan": "tua", 
             "banpres": "bantuan presiden", "bansarkas": "bantuan sarana kesehatan", "bazis": "badan amal, zakat, infak, dan sedekah",
             "bcoz": "karena", "beb": "sayang", "bejibun": "banyak", "belom": "belum", "bener": "benar", "ber2": "berdua",
             "berdikari": "berdiri di atas kaki sendiri", "bet": "banget", "beti": "beda tipis", "beut": "banget", "bgd": "banget", 
             "bgs": "bagus", "bhubu": "tidur", "bimbuluh": "bimbingan dan penyuluhan", "bisi": "kalau-kalau", "bkn": "bukan",
             "bl": "beli", "blg": "bilang", "blm": "belum", "bls": "balas", "bnchi": "benci", "bngung": "bingung", "bnyk": "banyak",
             "bohay": "badan aduhai", "bokep": "porno", "bokin": "pacar", "bole": "boleh", "bolot": "bodoh", "bonyok": "ayah ibu",
             "bpk": "bapak", "brb": "segera kembali", "brngkt": "berangkat", "brp": "berapa", "brur": "saudara laki-laki", "bsa": "bisa", 
             "bsk": "besok",'daring' :'online',"bu_bu": "tidur", "bubarin": "bubarkan", "buber": "buka bersama", "bujubune": "luar biasa", 
             "buser": "buru sergap", "bwhn": "bawahan", "byar": "bayar", "byr": "bayar", "c8": "chat", "cabut": "pergi", 
             "caem": "cakep", "cama-cama": "sama-sama", "cangcut": "celana dalam", "cape": "capek", "caur": "jelek",
             "cekak": "tidak ada uang", "cekidot": "coba lihat", "cemplungin": "cemplungkan", "ceper": "pendek",
             "ceu": "kakak perempuan", "cewe": "cewek", "cibuk": "sibuk", "cin": "cinta", "ciye": "cie", "ckck": "ck",
             "clbk": "cinta lama bersemi kembali", "cmpr": "campur", "cnenk": "senang", "congor": "mulut", "cow": "cowok", 
             "coz": "karena", "cpa": "siapa", "gokil": "gila", "gombal": "suka merayu", "gpl": "tidak pakai lama", 
             "gpp": "tidak apa-apa", "gretong": "gratis", "gt": "begitu", "gtw": "tidak tahu", "gue": "saya", 
             "guys": "teman-teman", "gws": "cepat sembuh", "haghaghag": "tertawa", "hakhak": "tertawa", 
             "handak": "bahan peledak", "hansip": "pertahanan sipil", "hellow": "halo", "helow": "halo", 
             "hi": "hai", "hlng": "hilang", "hnya": "hanya", "houm": "rumah", "hrs": "harus", "hubad": "hubungan angkatan darat", 
             "hubla": "perhubungan laut", "huft": "mengeluh", "humas": "hubungan masyarakat", "idk": "saya tidak tahu", 
             "ilfeel": "tidak suka", "imba": "jago sekali", "imoet": "imut", "info": "informasi", "itung": "hitung", 
             "isengin": "bercanda", "iyala": "iya lah", "iyo": "iya", "jablay": "jarang dibelai", "jadul": "jaman dulu", 
             "jancuk": "anjing", "jd": "jadi", "jdikan": "jadikan", "jg": "juga", "jgn": "jangan", "jijay": "jijik", 
             "jkt": "jakarta", "jnj": "janji", "jth": "jatuh", "jurdil": "jujur adil", "jwb": "jawab", "ka": "kakak", 
             "kabag": "kepala bagian", "kacian": "kasihan", "kadit": "kepala direktorat", "kaga": "tidak", "kaka": "kakak", 
             "kamtib": "keamanan dan ketertiban", "kamuh": "kamu", "kamyu": "kamu", "kapt": "kapten", "kasat": "kepala satuan", 
             "kasubbid": "kepala subbidang", "kau": "kamu", "kbar": "kabar", "kcian": "kasihan", "keburu": "terlanjur", 
             "kedubes": "kedutaan besar", "kek": "seperti", "keknya": "kayaknya", "keliatan": "kelihatan", "keneh": "masih", 
             "kepikiran": "terpikirkan", "kepo": "mau tahu urusan orang", "kere": "tidak punya uang", "kesian": "kasihan", 
             "ketauan": "ketahuan", "keukeuh": "keras kepala", "khan": "kan", "kibus": "kaki busuk", "kk": "kakak", "klian": "kalian", 
             "klo": "kalau", "kluarga": "keluarga", "klwrga": "keluarga", "kmari": "kemari", "kmpus": "kampus", "kn": "kan", 
             "knl": "kenal", "knpa": "kenapa", "kog": "kok", "kompi": "komputer", "komtiong": "komunis Tiongkok", 
             "konjen": "konsulat jenderal", "koq": "kok", "kpd": "kepada", "kptsan": "keputusan", "krik": "garing",
             "krn": "karena", "ktauan": "ketahuan", "ktny": "katanya", "kudu": "harus", "kuq": "kok", "ky": "seperti", 
             "kykny": "kayanya", "laka": "kecelakaan", "lambreta": "lambat", "lansia": "lanjut usia", "lapas": "lembaga pemasyarakatan",
             "lbur": "libur", "lekong": "laki-laki", "lg": "lagi", "lgkp": "lengkap", "lht": "lihat", "linmas": "perlindungan masyarakat",
             "lmyan": "lumayan", "lngkp": "lengkap", "loch": "loh", "lol": "tertawa", "lom": "belum", "loupz": "cinta", 
             "lowh": "kamu", "lu": "kamu", "luchu": "lucu", "luff": "cinta", "luph": "cinta", "lw": "kamu", "lwt": "lewat", 
             "maaciw": "terima kasih", "mabes": "markas besar", "macem-macem": "macam-macam", "madesu": "masa depan suram",
             "maen": "main", "mahatma": "maju sehat bersama", "mak": "ibu", "makasih": "terima kasih", "malah": "bahkan", 
             "malu2in": "memalukan", "mamz": "makan", "manies": "manis", "mantep": "mantap", "markus": "makelar kasus", 
             "mba": "mbak", "mending": "lebih baik", "mgkn": "mungkin", "mhn": "mohon", "miker": "minuman keras", 
             "milis": "mailing list", "mksd": "maksud", "mls": "malas", "mnt": "minta", "moge": "motor gede", 
             "mokat": "mati", "mosok": "masa", "msh": "masih", "mskpn": "meskipun", "msng2": "masing-masing", 
             "muahal": "mahal", "muker": "musyawarah kerja", "mumet": "pusing", "muna": "munafik", 
             "munaslub": "musyawarah nasional luar biasa", "musda": "musyawarah daerah", "muup": "maaf", "muuv": "maaf",
             "nal": "kenal", "nangis": "menangis", "naon": "apa", "napol": "narapidana politik", "naq": "anak", 
             "narsis": "bangga pada diri sendiri", "nax": "anak", "ndak": "tidak", "ndut": "gendut", 
             "nekolim": "neokolonialisme", "nelfon": "menelepon", "ngabis2in": "menghabiskan", 
             "ngakak": "tertawa", "ngambek": "marah", "ngampus": "pergi ke kampus", "ngantri": "mengantri", "ngapain": "sedang apa", 
             "ngaruh": "berpengaruh", "ngawur": "berbicara sembarangan", "ngeceng": "kumpul bareng-bareng", "ngeh": "sadar", 
             "ngekos": "tinggal di kos", "ngelamar": "melamar", "ngeliat": "melihat", "ngemeng": "bicara terus-terusan", 
             "ngerti": "mengerti", "nggak": "tidak", "ngikut": "ikut", "nginep": "menginap", "ngisi": "mengisi", 
             "ngmg": "bicara", "ngocol": "lucu", "ngomongin": "membicarakan", "ngumpul": "berkumpul", "ni": "ini", 
             "nyasar": "tersesat", "nyariin": "mencari", "nyiapin": "mempersiapkan", "nyiram": "menyiram", 
             "nyok": "ayo", "o/": "oleh", "ok": "ok", "priksa": "periksa", "pro": "profesional", 
             "psn": "pesan", "psti": "pasti", "puanas": "panas", "qmo": "kamu", "qt": "kita",
             "rame": "ramai", "raskin": "rakyat miskin", "red": "redaksi", "reg": "register", "rejeki": "rezeki",
             "renstra": "rencana strategis", "reskrim": "reserse kriminal", "sni": "sini", "somse": "sombong sekali", 
             "sorry": "maaf", "sosbud": "sosial-budaya", "sospol": "sosial-politik", "sowry": "maaf", "spd": "sepeda", 
             "sprti": "seperti", "spy": "supaya", "stelah": "setelah", "subbag": "subbagian", "sumbangin": "sumbangkan", 
             "sy": "saya", "syp": "siapa", "tabanas": "tabungan pembangunan nasional", "tar": "nanti", "taun": "tahun",
             "tawh": "tahu", "tdi": "tadi", "te2p": "tetap", "tekor": "rugi", "telkom": "telekomunikasi", "telp": "telepon", 
             "temen2": "teman-teman", "tengok": "menjenguk", "terbitin": "terbitkan", "tgl": "tanggal", "thanks": "terima kasih", 
             "thd": "terhadap", "thx": "terima kasih", "tipi": "TV", "tkg": "tukang", "tll": "terlalu", "tlpn": "telepon", "tman": "teman", "tmbh": "tambah", "tmn2": "teman-teman", "tmph": "tumpah", "tnda": "tanda", "tnh": "tanah", "togel": "toto gelap", "tp": "tapi", "tq": "terima kasih", "trgntg": "tergantung", "trims": "terima kasih", "cb": "coba", "y": "ya", "munfik": "munafik", "reklamuk": "reklamasi", "sma": "sama", "tren": "trend",
             "ngehe": "kesal", "mz": "mas", "analisise": "analisis", "sadaar": "sadar", "sept": "september", "nmenarik": "menarik",
             "zonk": "bodoh", "rights": "benar", "simiskin": "miskin", "ngumpet": "sembunyi", "hardcore": "keras", "akhirx": "akhirnya",
             "solve": "solusi", "watuk": "batuk", "ngebully": "intimidasi", "masy": "masyarakat", "still": "masih", "tauk": "tahu", 
             "mbual": "bual", "tioghoa": "tionghoa", "ngentotin": "senggama", "kentot": "senggama", "faktakta": "fakta",
             "sohib": "teman", "rubahnn": "rubah",'bdk2':'anak anak', "trlalu": "terlalu", "nyela": "cela", "heters": "pembenci", 
             "nyembah": "sembah", "most": "paling", "ikon": "lambang", "light": "terang", "pndukung": "pendukung",
             "setting": "atur", "seting": "akting", "next": "lanjut", "waspadalah": "waspada", "gantengsaya": "ganteng", 
             "parte": "partai", "nyerang": "serang", "nipu": "tipu", "ktipu": "tipu", "jentelmen": "berani", "buangbuang": "buang",
             "tsangka": "tersangka", "kurng": "kurang", "ista": "nista", "less": "kurang", "koar": "teriak", "paranoid": "takut",
             "problem": "masalah", "tahi": "kotoran", "tirani": "tiran", "tilep": "tilap", "happy": "bahagia", "tak": "tidak",
             "penertiban": "tertib", "uasai": "kuasa", "mnolak": "tolak", "trending": "trend", "taik": "tahi", "wkwkkw": "tertawa", 
             "ahokncc": "ahok", "istaa": "nista","aku":"saya",
             "benarjujur": "jujur", "mgkin": "mungkin",'tlp':'telepon','onlineotak':'otak daring','oflen':'luring'  
         ,'cancel':'batal','drpd':'daripada','tgs':'tugas','workfromhome':'bekerja dari rumah','atuser':'','emg':'emang','and':'dan','dha':'dia','kira2':'kira kira','cpt2':'cepat cepat','gmn':'gimana','yg':'yang','dpt':'dapat','jg':'juga','tmn':'teman','tmn2':'teman-teman','tdr':'tidur','byk':'banyak','bgt':'banget','bgtt':'banget','msi':'masih','msh':'masih','emg':'emang','mtk':'matematika','mtkk':'matematika','mtkkk':'matematika','kl':'kalau','ortuku':'orang tuaku','mrk':'mereka','ytb':'youtube','cb':'coba','krn':'karena','hrs':'harus','ndak':'tidak','tiba2':'tiba-tiba','hrs' : 'harus','ak' : 'aku','begimanaaa':'bagaimana','yg' : 'yang','laptoo' : 'laptop','gaada' : 'tidak ada','trus' : 'terus','onlen':'online','ngapsen' : 'mengabsen','dr':'dari','dpt':'dapat','bisa2':'bisa-bisa','yg':'yang','menyulidkan':'menyulitkan','dadi':'jadi','ak':'aku','ajg':'anjing','lg':'lagi','mangstap':'mantap','terossss':'terus','pembeljrn':'pembelajaran','nanya':'tanya','tny':'tanya','tp':'tapi','sblm':'seblum','lgsg':'langsung','ttg':'tentang','oplen':'online','drpd':'daripada','rmh':'rumah','slrh':'seluruh','ketum':'ketua umum','adkel':'adik kelas','mntp':'mantap','r i p':'mati','dahlah':'udahlah','ckuplah':'cukuplah','trs':'terus','kulon':'kuliah online', 'smt':'semester','pulang2':'pulang-pulang','cororong':'corona','notip':'notifikasi','ambis':'ambisi','onlennnn':'online','sm':'sama','ak':'aku','bgt':'banget','mls':'malas','dr':'dari','matlis':'mati listrik','tementemen':'teman-teman', 'udeh':'udah','grgr':'gara gara','jahat2':'jahat jahat','aq':'aku','kulyah':'kuliah','ad':'ada','school':'sekolah','nyusahke':'nyusahkan','irit2':'irit irit','mudeng':'ngerti','paham2':'paham paham','ak':'aku','jingan':'bajingan','ato':'atau','anj':'anjing','mless':'malas','oplen':'offline','class':'kelas','sering2':'sering sering','skul':'sekolah','kelar2':'kelar kelar','gr':'gara','guru2':'guru guru','berbeda2':'berbeda-beda','syg':'sayangku','pinter2':'pinter-pinter','tipsen':'titip absen','gblggg':'goblok','gblg':'goblok','anak2':'anak anak','slm':'selama','anjg':'anjing','pgn':'pengen','oflen':'offline','lama2':'lama-lama','males banget':'sangat malas','mles':'males','teman2':'teman-teman','ajggg':'anjing','ajg':'anjing','ak':'aku','bpk':'bapak','tida':'tidak','bener2':'bener bener','lgi':'lagi','paling burung':'paling buruk','kelen':'kalian','jgn':'jangan','stgh':'setengah','w':'gw','kyk':'kayak','org2':'orang-orang',"bet" : 'bener',"mangat" : "semangat","yg":"yang","sadgurl" : 'perempuan' , "smbil" : "sambil","barbar" : "biadab","ajigggg" : "anjing" , "apaan" : "apa" , "mlooo" : "mulu" ,"grgr": "gara-gara","smt":"semesterr","ttp" :"tetap" ,"yok" : "yuk", "beritane" : "beritanya","nurut2" : "nurut-nurut" ,"wa" : 'wassap', "mapel" : "mata pelajaran","ni" : "ini" , "kek" : "kayak","bosen" : "bosan" , "capek" : "cape" ,"bego" : "bodoh" , "kagak" : "ngga" , "kesel" : "kesal", 'oengen' : 'pengen', 'tyda' : 'tidak', 'yh':'ya','ako' : 'aku', 'lebi' : 'lebih', 'de' : 'deh', 'onlineee' : 'online', 'jd' : 'jadi', 'banya':'banyak','grgr' : 'gara gara', 'ibuk' : 'ibu', 'ppl':'','dom':'domisili','sekaliii':"sekali",'sbg' :'sebagai','org' : 'orang','gblk': 'bodoh','mksd' : 'maksud','gmn':'gimana','sokusinya':'solusinya','anjim':'anjing','flishhh':'plis','kl':'kalau',"betulll" : "betul","yutub":"youtube" ,"twitt" :"twitter", "bisaa" : "bisa" , "dripada" : "dari pada" , "utk" : "untuk","gegara" :"gara-gara", "donlot" : "mengunduh" ,"w" : "aku" , "dah" : "sudah" , "naek" : "naik","abis" : "habis" , "otw" : "menuju" , "tp" : "tapi" , "bsk" : "besok" , "lg" : "lagi" , "rakan2" : "rekan-rekan" ,"mengharungi" : "mengarungi" , "fizikal" : 'fisik',"mangstap" : "mantap" , "bat" : "bener" ,"online" : "onlen" ,"yoi" : "ya" , "yooii" : "ya" , "beraaa" : "berasa" ,"bgt" : "banget","tpi":"tapi","klihatan" : "kelihatan" , "trus" : "terus" , "mksudny" : "maksudnya","gk" : "gak" , "gni" : "gini","us" : "ujian sekolah" , "utbk" : "ujian tulis berbasis komputer" , "ngalong" : "begadang" , "klo" : "kalau" , "please" : "tolong" , "dbuat" : "dibuat" ,"gmna" : "gimana" , "masup" : "masuk" , "nngys" : "nangis" , "goblo" : "bodoh" , "mnding" : "mending" , "aj" : "aja" , "dr" : "dari" ,"gwa" : "aku" , "jg" : "juga" , "tkt" : "takut","samsek" : "sama sekali" , "we" : "aku" , "anjir" : " " , "kerad" : "gila","lg" : "lagi", "tp" : "tapi", "cm" : "cuma", "bs" : "bisa", "prnh" : "pernah","g" : "nggak", "rl" : "real life", "krn" : "karena", "tkt" : "takut", "tmn" : "teman", "jg" : "juga", "hrs" : "harus", "lnjut" : "lanjut", "pjj" : "pembelajaran jarak jauh","trs" : "terus", "scr" : "secara", "nder" : "sender", "lg" : "lagi", "bgt" : "banget", "yg" : "yang", "skr" : "sekarang", "sekul" : "sekolah", "gtu" : "gitu", "gc" : "group chat","mhs" : "mahasiswa", "udh" : "udah", "kl" : "kalo", "gk" : "nggak", "bgst" : "bangsat", "jan" : "jangan", 'pjj': "pembelajaran jarak jauh" , "jan" : "jangan" , "emg" : "emang" ,"y" : "ya", "lu" : "kamu" , "gue" : "aku" , "gua" : "aku" ,"bkn" : "bukan" ,"classes" : "kelas" ,"blm" : "belum" , "classes" : "kelas" ,"blm" : "belum" ,"conference" : "konferensi" , "kadis" : "kepala dinas" , "kabid" : "kepala bidang" , "entah" : "tidak tahu" , "hbis" : "habis", "lupaa" : "lupa" ,"dahlah" : "sudahlah" , "tl" : "timeline" , "prefer" : "lebih memilih" , "gakuat" : "tidak kuat" , "ampunn" : "ampun" , "kampus2" : "kampus-kampus" ,"dpr" : "dewan perwakilan rakyat" , "rapotan" : "raport" , "ky" : "seperti" , "gmn" : "gimana" , "i" : "saya" , "mean" : "maksud" , "blas" : "sama sekali" ,"kasi" : "kasih" , "seneng" : "senang" , "flis" : "tolong" , "gakuad" : "tidak kuat" , "bk" : "besok" , "bbrp" : "beberapa" , "hate" : "benci" ,   "gabut" : "bosan" , "nungguin" : "menunggu" , "nyelesain" : "menyelesaikan" , "pengen" : "mau" ,"skolah" : "sekolah" ,'wth':'apa apaan', "ntar" : "nanti" , "satu2nya" : "satu-satunya" , "ngelawan" : "melawan" , "ena" : "enak" , "statistics" : "statistik" ,"manusya" : "manusia" , "kwad" : "kuat" , "cumanan" : "hanya" , "tmptku" : "tempatku" , "sm" : "sama" , "thx" : "terimakasih" ,"ngapainnnn" : "ngapain" , "lord" : "tuan" , "masi" : "masih" , "nyari" : "mencari" , "gx" : "tidak" , "km" : "kamu" , "gabisa" : "tidak bisa" , "langsungg" : "langsung" , "gada" : "tidak ada" , "kbnykn" : "kebanyakan" , "gtu" : "gitu" , "sfh" : "belajar dari rumah" , "smngt" : "semangat" ,"rewelnya" : "susahnya" ,"sdg" : "sedang" , "thun" : "tahun" , "pemkot" : "pemerintah kota" , "ppdb" : "penerimaan peserta didik baru" , "nakes" : "tenaga kesehatan" ,"lipia" : "lembaga ilmu pengetahuan islam dan arab" , "mulu" : "terus" , "ngingetin" : "mengingatkan" , "unk" : "untuk" , "makasihh" : "terimakasih" ,"td" : "tadi" , "suruu" : "suruh" , "reward" : "penghargaan" , "yak" : "iya" , "un" : "ujian nasional" ,  "ipk" : "indeks prestasi kumulatif" , "iyh" : "iya" , "ngx" : "tidak" , "mo" : "mau" , "pls" : "tolong" , "gatahan" : "tidak tahan" , "msih" : "masih" , "day" : "hari" , "karna" : "karena" , "mageran" : "malas gerak" , "nerapin" : "menerapkan" , "guru2" : "guru-guru" , "cuman" : "hanya" , "betuls" : "betul" , "stay" : "tinggal" , "krucil2" : "anak-anak" , "ngurusinnya" : "mengurusnya" , "matkul" : "mata kuliah" , "gaada" : "tidak ada" , "jl" : "jalan" , "jakpus" : "jakarta pusat" , "for" : "untuk" , "education" : "pendidikan" , "boro" : "jangankan" , "or" : "atau" , "ges" : "guys" , "ad" : "ada" , "muk" : "mau" ,"graphic" : "grafik" , "gaaaa" : "tidak" , "teross" : "terus" , "break" : "istirahat" , "ttep" : "tetap" , "hyung" : "kakak" , "good night" : "selamat malam" , "medsos" : "media sosial" , "tgl" : "tanggal" , "mrah" : "merah" , "ldr" : "hubungan jarak jauh" ,"jd" : "jadi" , "seabreg" : "banyak" ,  "ipk" : "indeks prestasi kumulatif" , "iyh" : "iya" , "ngx" : "tidak" , "mo" : "mau" , "pls" : "tolong" , "gatahan" : "tidak tahan" , "msih" : "masih" , "day" : "hari" , "karna" : "karena" , "mageran" : "malas gerak" ,"gatidur" : "tidak tidur" ,"pantes" : "pantas" , "tdi" : "tadi" , "baek" : "baik" , "berb" : "verb" ,  "bo" : "mau" , "netthink" : "nethink" , "ama" : "sama" , "cicle" : "lingkungan" , "ma" : "sama" , "kaayak" : "seperti" , "yaallah" : "ya allah" , "laen" : "lain" , "nih" : "ini" , "spertinya" : "sepertinya" , "tida" : "tidak" , "kdg" : "kadang" , "b" : "bahasa" , "mauuu" : "mau" , "ato" : "atau" , "apk" : "aplikasi" , "pebisnis" : "pembisnis" , "opo" : "apa" , "ora" : "tidak" , "sabaarrrr" : "sabar" , "jdi" : "jadi" , "tp" : "tapi" , "prepare" : "mempersiapkan" , "halokes" : "sekolah" , "bobo" : "tidur" , "after" : "setelah" , "class" : "kelas" , "cape" : "lelah" , "ngajar" : "mengajar" , "kndisi" : "kondisi" , "spt" : "seperti" , "yth" : "yang terhormat" , "brkontribusi" : "berkontribusi" , "mmberikan" : "memberikan" , "mmbuat" : "membuat" , "tmpt" : "tempat" , "adik2" : "adik-adik" , "skrg" : "sekarang" ,"bnyk" : "banyak" , "blkngan" : "belakangan" , "ttg" : "tentang" , "msk" : "masuk" , "dng" : "dengan" , "mama" : "ibu" , "teacher" : "guru" , "tmn" : "teman" , "mtk" : "matematika" , "met" : "selamat" , "slamat" : "selamat" , "well" : "jadi" , "ampe" : "sampai" , "mantep" : "mantap" , "ujung2nya" : "akhirnya" , "sudahhh" : "sudah" , "agsts" : "agustus" , "ipnya" : "indeks prestasi nya" , "udh" : "udah" , "krs" : "kartu rencana studi" , "smngat" : "semangat" , "5nya" : "5" ,"smg" : "semoga" , "nnti" : "nanti" , "gaenak" : "tidak enak" , "masup" : "masuk" , "ngulang" : "mengulang" , "itung" : "hitung" , "biar" : "supaya" , "kea" : "seperti" , "ak" : "aku" , "hrus" : "harus" , "ap" : "apa" , "dlu" : "dulu" , "gw" : "aku" ,"pake" : "pakai" , "bayangin" : "bayangkan" ,"bikin" : "buat" , "trs" : "terus" , "ujung" : "akhirnya" , "req" : "request" , "bobol" : "menjebol" ,"sikon" : "situasi dan kondisi" , "bnr2" : "benar-benar" , "ready" : "siap" ,"kids" : "anak-anak" , "must" : "harus" , "be safe" : "berhati-hati" , "bantuinn" : "bantuin" , "yaa" : "ya" , "pic" : "picture" ,"they" : "mereka" , "not" : "belum" , "a" : "menjadi seseorang" , "man" : "laki-laki dewasa" ,'ga':'tidak','iq':'aku','jkw':'jokowi', 'sbnrnya':'sebenarnya',"soale" : "soalnya" ,"assesment" : "penilaian" , "magang2" : "magang" , "bakalan" : "akan" , "mon maap" : "mohon maaf" , "kalo" : "jika" , "gajadi" : "tidak jadi" , "gakan" : "tidak akan" ,"da" : "lah" , "aing" : "saya" , "nyuwun sewu" : "maaf" , "pki" : "partai komunis indonesia" , "sy" : "saya" , "lbh" : "lebih" ,"dl" : "dulu" , "org" : "orang" , "anak2" : "anak-anak" ,"gara2" : "gara-gara" , "napa" : "kenapa" , "malming" : "malam minggu" , "tuh" : "itu" , "tetep" : "tetap" , "pdhal" : "padahal" , "skrang" : "sekarang" , "nggak" : "tidak" , "aja" : "saja" ,"sumpahlah" : "sumpah" , "rto" : "request time out" , "face to face" : "tatap muka" , "online class" : "kelas online" , "gara2" : "gara-gara" , "napa" : "kenapa" ,"kgk" : "tidak" , "dpt" : "dapat" ,"pliss" : "tolong" ,"mapel" : "mata pelajaran" ,"taknak" : "tidak mau" , "stress" : "stres" ,"gausah" : "tidak usah" , "tak" : "tidak" ,"ndek" : "di" , "sampek" : "sampai" , "hp" : "handphone" ,"gabut" : "bosan" , "ngesakno" : "menyusahkan" , "dekne" : "ngrsakno" ,"universiti" : "universitas" ,"ndilok" : "melihat" ,"ngga" : "tidak" , "jelimet" : "rumit" ,"cuba" : "coba" , "nak" : "mau" ,"bnr" : "benar" ,"kek" : "kayak" ,"belii" : "beli" ,"ndak" : "tidak" , "mashok" : "masuk" , "udaa" : "sudah" , "gedeee" : "besar" ,  "worth" : "bernilai" ,"baiknya" : "sebaiknya" , "dr" : "dari" ,"gwe" : "aku" , "from" : "dari" , "home" : "rumah" ,"mrka" : "mereka" , "utk" : "untuk" , "bs" : "bisa" ,"krn" : "karena" ,"g" : "tidak" ,  "mmf" : "maaf" , "klo" : "kalau" , "ad" : "ada" , "bsa" : "bisa" , "dipke" : "dipake" ,"gangerti" : "tidak mengerti" ,"apa2" : "apa-apa" ,"free" : "bebas" , "drmn" : "dari mana" , "rt" : "retweet" , "nsk" : "mau" ,  "cm" : "cuma" , "kayaknya" : "sepertinya" , "tu" : "itu" ,"gegara" : "gara-gara" , "sekaly" : "sekali" ,"btw" : "ngomong-ngomong" , "rada" : "agak" ,"bbrp" : "beberapa" , "nyantol" : "nyangkut" , "drpd" : "daripada" , "ga" : "tidak" , "taun" : "tahun" , "to" : "tryout" , "ptn" : "perguruan tinggi negeri" ,"bareng" : "bareng-bareng" , "semangatt" : "semangat" , "dlm" : "dalam" , "gesss" : "guys" , "smoga" : "semoga" , "la" : "lah" , "aq" : "aku" , "jg" : "juga" , "dgn" : "dengan" , "anjir" : " " ,"skuy" : "yuk" , "mending" : "lebih baik" , "yg" : "yang" ,"ato" : "atau" , "gugel" : "google" , "gamau" : "gak mau" ,"skrng" : "sekarang" ,  "sklh" : "sekolah" , "bgini" : "begini" , "lbh" : "lebih" , "bljr" : "belajar" ,"misqueen" : "miskin" ,"oengen" : "mau" , 'tyda' : "tidak" , "onlen" : "online" ,"menjadu" : "menajdi" , "vc" : "video call" ,  "capek" : "cape" ,"bego" : "bodoh" , "kagak" : "tidak" , "kesel" : "kesal", "mb" : " " ,"dgn" : "dengan" ,"spp" : "sumbangan pembinaan pendidikan" , "gegara" : "gara gara" , "ak" : "aku" , "mbolos" : "bolos" , "aq" : "aku" , "krena" : "karena" ,"nggk" : "enggak" , "ad" : "ada" , "yh" : "ya" , "bkan" : "bukan" , "diy" : "daerah istimewa yogyakarta" , "nder" : "brother" , "tq" : "thank you" , "manala" : "mana yang" ,"yaaa" : "ya" , "thn" : "tahun" , "jgn" : "jangan" ,"syg" : "sayang" , "nambah2" : "nambah-nambah" , "kpn" : "kapan" , "mumet" : "pusing" , "ges" : "guys" , "keja" : "kerja" , "akuu" : "aku" , "tuu" : "itu" , "gituu" : "gitu" , "hengpon" : "handphone" , "dll" : "dan lain-lain" , "murit" : "murid" ,"klayapan" : "kelayapan" , "ngilang" : "hilang" , "gilaaaaa" : "gila" , "ukt" : "uang kuliah tinggal" , "w" : "aku" ,"beritane" : "beritanya" , "nurut2" : "nurut-nurut" , "wa" : "whatsapp" , "mapel" : "mata pelajaran" , "ni" : "ini" , "kek" : "kayak" , "bosen" : "bosan" , "kgn" : "kangen" , "kls" : "kelas" ,"kpai" : "komisi perlindungan anak indonesia" , "gaes" : "guys" , "klian" : "kalian" , "mksdnya" : "maksudnya" , "gimna" : "gimana" ,"riweh" : "ribet" , "ancur" : "hancur" , "brou" : "bro" , "gaenak" : "tidak enak" , "smt" : "semester" , "ttp" :"tetap" ,"yok" : "yuk" ,"mlooo" : "mulu" , "grgr" : "gara-gara" , "smbil" : "sambil" , "barbar" : "biadab" ,"ajigggg" : "anjing" , "apaan" : "apa" ,"lg" : "lagi" , "rakan2" : "rekan-rekan" ,"mengharungi" : "mengarungi" , "fizikal" : "fisik" , "mangstap" : "mantap" , "bat" : "banget" , "bet" : "benar" ,"mangat" : "semangat" , "yg" : "yang" , "sadgurl" : 'perempuan' ,"donlot" : "mengunduh" , "w" : "aku" , "dah" : "sudah" , "naek" : "naik" ,"abis" : "habis" , "otw" : "menuju" , "bsk" : "besok" , "betulll" : "betul","yutub" :"youtube" , "twitt" : "twitter" , "bisaa" : "bisa" , "dripada" : "dari pada" , "utk" : "untuk","gegara" :"gara-gara","trus" : "terus" ,"beraaa" : "berasa" ,"bgt" : "banget" , "tpi" : "tapi" , "onlen" : "online" , "yoi" : "ya" , "yooii" : "ya" ,"gni" : "begini", "klihatan" : "kelihatan" , 'kmtllll':'kontol','pembelajaranjarakjauh':'pembelajaran jarak jauh', 'pjj': "pembelajaran jarak jauh" , "jan" : "jangan" , "emg" : "memang" ,"y" : "ya", "lu" : "kamu" , "gue" : "aku" , "gua" : "aku" , "bkn" : "bukan" , "mksudny" : "maksudnya","gk" : "gak"  }

    kamusAlay = pd.read_csv('https://raw.githubusercontent.com/nasalsabila/kamus-alay/master/colloquial-indonesian-lexicon.csv')
    stopWord = list(pd.read_csv('https://raw.githubusercontent.com/datascienceid/stopwords-bahasa-indonesia/master/stopwords_id_satya.txt', header = None)[0])
    freq_english = list(pd.read_csv('https://raw.githubusercontent.com/kaenova/GemastikXIV_Yaudahlah/main/data/unigram_freq.csv')[:500]['word'])

    @classmethod
    def DataCleaningBatch(cls, df):
        '''
        DataCleaningBatch will Clean Data from a Pandas Dataframe with a 'tweet' column and return a single column of Pandas DataFrame of Cleaned text.
        '''

        final_list = []

        for index, row in tqdm(df.iterrows(), total=df.shape[0]):
            text_temp = row['tweet']
            # Cleaning text
            text_temp = cls.CleanText(text_temp)
            # Menghilangkan Bahasa Inggris
            text_temp = cls.ApplyKamusAlayStopTranslate(text_temp)
            if text_temp != None:
                if len(text_temp) != 0:
                    final_list.append(text_temp)

        final_list = list(set(final_list))
        final_dict = {'tweet':final_list}
        new_df = pd.DataFrame(final_dict)
        return new_df

    @classmethod
    def CleanText(cls, text):
        text = re.sub(r'http\S+', '', text)
        text = re.sub('(@\w+|#\w+)','',text)
        #will replace the html characters with " "
        text=re.sub('<.*?>', '', text)  
        #To remove the punctuations
        text = text.translate(str.maketrans(' ',' ',string.punctuation))
        #will consider only alphabets
        text = re.sub('[^a-zA-Z]',' ',text) 
        #will replace newline with space
        text = re.sub("\n"," ",text)
        #will convert to lower case
        text = text.lower()
        # will replace a word
        text = re.sub("(username|user|url|rt|xf|fx|xe|xa)\s|\s(user|url|rt|xf|fx|xe|xa)","",text)
        # will repalce repated char
        text = re.sub(r'(\w)(\1{2,})', r"\1", text)
        # will replace single word
        text = re.sub(r"\b[a-zA-Z]\b","",text)
        # will replace space more than one
        text = re.sub('(s{2,})',' ',text)
        # will join the words
        text=' '.join(text.split())
        return text

    @classmethod
    def ApplyKamusAlayStopTranslate(cls, text):

        # split
        text = text.split(' ')

        for word in text:
            if word in cls.freq_english or word in cls.custom_delete_string:
                return None

        # apply kamusAlay
        temp=[]
        for i in range(len(text)):
            try:
                text[i] = cls.kamus[text[i]]
            except:
                pass

            try:
                # kamus alay
                index = cls.kamusAlay.index[kamusAlay['slang'] == text[i]][0]
                text[i] = cls.kamusAlay['formal'][index]
            # stop word
            except:
                pass

            try:
                # menghapus kata jika ada di list stopWord
                if (text[i] in cls.stopWord):
                    text.remove(text[i])
            except:
                pass

        text = ' '.join(text)
        return text
        

# Cleaning every CSV on a Foder

In [4]:
import os
directory = '/content/drive/Shareddrives/GEMASTIK XIV: Yaudahlah/data/validation/raw/without_replies_retweet' #@param {type:"string"}
out_directory = '/content/drive/Shareddrives/GEMASTIK XIV: Yaudahlah/data/validation/processed/without_replies_retweet' #@param {type:"string"}

if directory[len(directory)-1] != '/':
  directory = directory + '/'

if out_directory[len(out_directory)-1] != '/':
  out_directory = out_directory + '/'

files = os.listdir(directory)

for file in files:
    if file[len(file)-4:] == '.csv':
      print(file)
      df = pd.read_csv(directory+file)
      df_temp = DataCleaning.DataCleaningBatch(df)
      df_temp.to_csv(out_directory+file, index=False)

astrazenaca.csv


100%|██████████| 4322/4322 [00:01<00:00, 2870.24it/s]


berhasil.csv


100%|██████████| 8902/8902 [00:04<00:00, 2106.98it/s]


berbiaya.csv


...

100%|██████████| 4049/4049 [00:01<00:00, 3035.42it/s]


sinovac.csv


100%|██████████| 7053/7053 [00:02<00:00, 2480.52it/s]


sinopharm.csv


100%|██████████| 642/642 [00:00<00:00, 2272.36it/s]


# Cleaning Testing

In [5]:
df_test = pd.read_csv('/content/drive/Shareddrives/GEMASTIK XIV: Yaudahlah/data/validation/processed/(covid OR corona OR covid19 OR covid-19 OR ) (vaksinasi OR vaksin OR vaksinisasi) ("tidak berguna" OR gagal OR mematikan OR merugikan OR rugi) -filter:links -filter:retweets -filter:replies.csv')

In [20]:
import re
test_text = 'jelas ini adalah kuncinya'
list_blocked_word = ['je']

for word in list_blocked_word:
  print('Checking on word: {}'.format(word))
  print(re.search(word, test_text))

Checking on word: je
<re.Match object; span=(0, 2), match='je'>


In [48]:
!rm -f -r '/content/drive/Shareddrives/GEMASTIK XIV: Yaudahlah/data/validation/processed/'*'.csv'

In [78]:
'immunity' in (list(pd.read_csv('https://raw.githubusercontent.com/kaenova/GemastikXIV_Yaudahlah/main/data/unigram_freq.csv')[:5000]['word']))

False